# Estudo sobre a base do jogo Jokenpower

In [150]:
import numpy as np
import pandas as pd

df = pd.read_csv('attacks_pattern.csv')
df.head()

,timestamp,time_since_start,damaged_player,attack_type,damage,is_counter_attack
0,2023-03-21 21:52:06.619274,2630,Enemy,Rock,10,False
1,2023-03-21 21:52:07.946170,3956,Enemy,Paper,20,True
2,2023-03-21 21:52:08.667869,4678,Enemy,Paper,1,False
3,2023-03-21 21:52:10.803697,6814,Enemy,Rock,20,True
4,2023-03-21 21:52:11.836542,7847,Enemy,Rock,10,False


## Tratamento de dados

In [151]:
# Parsear string timestamp para Tipo Timestamp do pandas
df['timestamp'] = pd.to_datetime(df['timestamp'], infer_datetime_format=True)

In [152]:
# Criar campo com diferença de tempo entre ataques
df['time_diff'] = df['timestamp'].diff().dt.total_seconds()

In [153]:
# Definir início de partida: quando próximo time_since_start for menor que anterior
df.loc[df['time_since_start'].diff() < 0, 'time_diff'] = np.nan

In [154]:
# Deletar NaN da tabela
df.dropna(inplace=True)

In [155]:
# Normalizando diferença de tempo entre ataques
df['label_time_diff'] = df['time_diff'].apply(lambda x: int(x*10))

In [156]:
# Normalizando valores qualitativos para quantitativos

def get_norm_attack_type(atk: str):
    result = 99
    if atk == 'Rock':
        result = 0
    elif atk == 'Paper':
        result = 1
    elif atk == 'Scissors':
        result = 2
    return result

df['label_attack_type'] = df['attack_type'].apply(get_norm_attack_type)

In [157]:
df.head()

,timestamp,time_since_start,damaged_player,attack_type,damage,is_counter_attack,time_diff,label_time_diff,label_attack_type
1,2023-03-21 21:52:07.946170,3956,Enemy,Paper,20,True,1.326896,13,1
2,2023-03-21 21:52:08.667869,4678,Enemy,Paper,1,False,0.721699,7,1
3,2023-03-21 21:52:10.803697,6814,Enemy,Rock,20,True,2.135828,21,0
4,2023-03-21 21:52:11.836542,7847,Enemy,Rock,10,False,1.032845,10,0
5,2023-03-21 21:52:13.783248,9794,Main,Paper,1,False,1.946706,19,1


## Scikit Learn

In [158]:
# Pegar apenas danos ao inimigo = Apenas do player
df_only_player = df.loc[df['damaged_player'] == 'Enemy'].copy()
df_only_player.reset_index(inplace=True)

In [159]:
# Pegar 80% da base para usar para treinamento
df_training = df_only_player[df_only_player.index < np.percentile(df_only_player.index, 80)]

# Pegar 20% restante para validação
df_validation = df_only_player[df_only_player.index >= np.percentile(df_only_player.index, 80)].copy()
df_validation.reset_index(inplace=True)

In [160]:
# Filtrar apenas colunas necessárias para fator X
x_chance_contra_ataque = df_training.loc[:, ['label_attack_type', 'label_time_diff']].values

In [161]:
# Filtrar apenas coluna necessária para fator Y
y_chance_contra_ataque = df_training.loc[:, 'is_counter_attack'].values

### Treinamento do modelo com algoritmo Naive Bayes

In [162]:
from sklearn.naive_bayes import GaussianNB

naive_chance_contra_ataque = GaussianNB()
naive_chance_contra_ataque.fit(x_chance_contra_ataque, y_chance_contra_ataque)

GaussianNB()

### Previsão

In [163]:
# utilizando o modelo: de acordo com o input de teste, prever se irá contra-atacar
# [[0, 7]] -> [['Rock', time_diff]]
previsao = naive_chance_contra_ataque.predict([[0, 7]])
previsao

array([False])

# Validação do Algoritmo

In [164]:
# Criando coluna de predições de contra-ataques do bot
# Para o input no algoritmo de predição é necessário utilizar os campos parâmetro para a predição. Neste caso: ['label_attack_type', 'label_time_diff']
parametros_para_contra_ataque = df_validation.loc[:, ['label_attack_type', 'label_time_diff']].values

# Usar 'parametros_para_contra_ataque' como input no algoritmo
nb_is_counter_attack = naive_chance_contra_ataque.predict(parametros_para_contra_ataque)

# Add na tabela base
df_validation['nb_is_counter_attack'] = nb_is_counter_attack
df_validation.head()

,level_0,index,timestamp,time_since_start,damaged_player,attack_type,damage,is_counter_attack,time_diff,label_time_diff,label_attack_type,nb_is_counter_attack
0,577,840,2023-03-21 22:13:47.798101,169195,Enemy,Scissors,20,True,1.462789,14,2,True
1,578,841,2023-03-21 22:13:48.529459,169926,Enemy,Scissors,3,False,0.731358,7,2,False
2,579,843,2023-03-21 22:13:51.138592,172535,Enemy,Rock,10,False,1.047484,10,0,False
3,580,845,2023-03-21 22:13:53.216550,174613,Enemy,Scissors,3,False,0.884969,8,2,False
4,581,847,2023-03-21 22:13:55.671725,177069,Enemy,Rock,10,False,1.092209,10,0,False


In [165]:
# comparar valores iguais entre 'input do usuário' vs 'gerado pelo Naive Bayes'
df_validation['comparison'] = df_validation['is_counter_attack'] == df_validation['nb_is_counter_attack']
df_validation['comparison'].value_counts(normalize=True) * 100

True     92.413793
False     7.586207
Name: comparison, dtype: float64